# Logistic Regression (using sklearn and manually build via PyTorch)

1. Using sklearn library

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [2]:
# Read data
df = pd.read_csv('ChurnData.csv')
df.head()

,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,...,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,4.40,...,1.0,0.0,1.0,1.0,0.0,1.482,3.033,4.913,4.0,1.0
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,9.45,...,0.0,0.0,0.0,0.0,0.0,2.246,3.240,3.497,1.0,1.0
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,6.30,...,0.0,0.0,0.0,1.0,0.0,1.841,3.240,3.401,3.0,0.0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,6.05,...,1.0,1.0,1.0,1.0,1.0,1.800,3.807,4.331,4.0,0.0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,7.10,...,0.0,0.0,1.0,1.0,0.0,1.960,3.091,4.382,3.0,0.0


In [3]:
data = df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip', 'churn']]
data.head()

,tenure,age,address,income,ed,employ,equip,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,1.0
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,0.0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,0.0


In [4]:
# Extract to X, y
X = data[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip']].values
y = data['churn'].values

In [5]:
# Feature scaling & normalize
X = StandardScaler().fit_transform(X)

In [6]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

In [7]:
# Create model
lr_sk = LogisticRegression(C=0.01, solver='liblinear')
lr_sk.fit(X_train, y_train)

LogisticRegression(C=0.01, solver='liblinear')

In [8]:
# Predict
y_hat_sk_prob = lr_sk.predict_proba(X_test)
print('Log loss (SK): %.3f' % log_loss(y_test, y_hat_sk_prob)) 

Log loss (SK): 0.602


2. Build model manually via PyTorch

In [9]:
# Import libraries
import torch
import torch.nn as nn

In [10]:
# Convert data to tensors
X_train_pt = torch.from_numpy(X_train.astype(np.float32))
y_train_pt = torch.from_numpy(y_train.astype(np.float32))
X_test_pt = torch.from_numpy(X_test.astype(np.float32))
y_test_pt = torch.from_numpy(y_test.astype(np.float32))

In [11]:
# Reshape y
y_train_pt = y_train_pt.view(y_train_pt.shape[0], 1)
y_test_pt = y_test_pt.view(y_test_pt.shape[0], 1)

In [12]:
print('X\'s shape:', list(X_train_pt.shape))
print('y\'s shape:', list(y_train_pt.shape))

X's shape: [160, 7]
y's shape: [160, 1]


In [13]:
# Create a Logistic Regression class
class pt_LogisticRegression(nn.Module):
    def __init__(self, input_size):
        super(pt_LogisticRegression, self).__init__()
        self.lin = nn.Linear(input_size, 1)
        
    def forward(self, x):
        z = self.lin(x)
        y_hat = torch.sigmoid(z)
        return y_hat

In [14]:
# Create model
def pt_model(X, y, epsilon=1e-25, learning_rate=0.01, show_loss=False):
    prev_loss = 0
    epoch = 0
    
    X_shape = list(X.shape)
    y_shape = list(y.shape)
    
    # Initialize model, loss function and optimizer
    model = pt_LogisticRegression(X_shape[1])
    loss_func = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    
    # Training
    while True:
        # Forward pass
        y_hat = model(X)
        
        # Loss function
        loss = loss_func(y_hat, y)
        
        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        # Show loss
        if show_loss:
            if (epoch+1) % 100 == 0:
                print('Epoch:', epoch+1, end='; ')
                print('Loss: %.3f' % loss.item())
                
        # Exit when loss distance is below epsilon
        if torch.abs(prev_loss - loss) < epsilon and epoch >= 1: break
        
        optimizer.zero_grad()
        prev_loss = loss.item()
        epoch += 1
        
    return model

In [15]:
lr_pt = pt_model(X_train_pt, y_train_pt)

In [16]:
# Predict
def pt_predict_proba(X, model):
    z = model(X)
    z = z.detach() # for disabling grad
    y_hat = torch.sigmoid(z)
    return y_hat

In [17]:
y_hat_pt_prob = pt_predict_proba(X_test_pt, lr_pt)
print('Log loss (PT): %.3f' % log_loss(y_test_pt.numpy(), y_hat_pt_prob.numpy()))

Log loss (PT): 0.698
